In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import csv
import numpy as np
import SD2_Model as SD

In [7]:

#net = SD.Net()
device = torch.device("cuda") 
net = SD.Net().to(device)
summary(net, input_size = (1,2))
#print(net)

loss_function = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 5]              15
            Linear-2                 [-1, 1, 5]              30
            Linear-3                 [-1, 1, 5]              30
            Linear-4                 [-1, 1, 5]              30
            Linear-5                 [-1, 1, 2]              12
Total params: 117
Trainable params: 117
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [8]:
#dummy data
#trainset = (torch.rand(2,1, device="cuda"), torch.rand(2,1, device="cuda") ) #Xs and Ys
trainset = torch.tensor([[1,2],[3,4]]).to(device)
print(trainset)

tensor([[1, 2],
        [3, 4]], device='cuda:0')


In [9]:
Load_data = True
dataset_path = "../2NN_dataset_flight.3.csv"
if(Load_data):
    trainset = []
    with open(dataset_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            #feature, data_label = int(row[0:2]),row[2:4]
            feature, data_label = (int(row[0]),int(row[1])),(float(row[2]) * 100 ,float(row[3]) * 100) #convert to cm
            #feature, data_label = (int(row[0]),int(row[1])),(round(float(row[2]) * 100) ,round(float(row[3]) * 100)) #convert to cm
            trainset.append([feature, data_label])
            #print([feature, data_label])

trainset = torch.Tensor(trainset).to(device)
#print(trainset)

test_data = True
if(test_data):
    #trainset = np.array(trainset)
    #print(trainset[1])
    feature, data_label = trainset[0]
    print(feature, data_label)
    #print(feature.view(1,2))
    output = net(feature)
    print(output)



tensor([253., 151.], device='cuda:0') tensor([ 0.4324, -2.2259], device='cuda:0')
tensor([-0.4666,  0.3340], device='cuda:0', grad_fn=<AddBackward0>)


In [10]:
train = True
epochs = 100
best_loss = 0.0
if (train):
    net.train()
    for epoch in range(epochs):
        for data in trainset:  
            feature, data_label = data #
            #print(data)
            #print(feature)
            #print(data_label)
            net.zero_grad()  
            output = net(feature)
            loss = loss_function(output, data_label) 
            loss.backward() 
            optimizer.step() 
        print(epoch,loss)
        if(epoch == 1):
            best_loss = loss.item()
        if(loss.item() < best_loss or loss.item() < 1.0):
            best_loss = loss.item()
            best_save = "./models_saves/best_2nd_NN_"
            torch.save(net, best_save + str(epoch) + "_" + str(round(best_loss,5)) +".pt")
    print(best_loss)

0 tensor(518.8115, device='cuda:0', grad_fn=<MseLossBackward>)
1 tensor(140.6212, device='cuda:0', grad_fn=<MseLossBackward>)
2 tensor(30.9913, device='cuda:0', grad_fn=<MseLossBackward>)
3 tensor(2.0027, device='cuda:0', grad_fn=<MseLossBackward>)
4 tensor(12.5325, device='cuda:0', grad_fn=<MseLossBackward>)
5 tensor(13.0900, device='cuda:0', grad_fn=<MseLossBackward>)
6 tensor(13.0536, device='cuda:0', grad_fn=<MseLossBackward>)
7 tensor(12.5798, device='cuda:0', grad_fn=<MseLossBackward>)
8 tensor(9.3403, device='cuda:0', grad_fn=<MseLossBackward>)
9 tensor(8.5636, device='cuda:0', grad_fn=<MseLossBackward>)
10 tensor(7.2635, device='cuda:0', grad_fn=<MseLossBackward>)
11 tensor(9.3594, device='cuda:0', grad_fn=<MseLossBackward>)
12 tensor(11.0619, device='cuda:0', grad_fn=<MseLossBackward>)
13 tensor(10.4241, device='cuda:0', grad_fn=<MseLossBackward>)
14 tensor(9.2326, device='cuda:0', grad_fn=<MseLossBackward>)
15 tensor(7.4865, device='cuda:0', grad_fn=<MseLossBackward>)
16 tens

In [11]:
#save network
save = True
#save = False
model_save_path = "2nd_NN.pt"
if (save):
    torch.save(net, model_save_path)

In [12]:
#test evaluate
dataset_path = "../2NN_dataset_flight.2.csv"
if(False):
    net.eval()
    eval_data = []
    with open(dataset_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            #feature, data_label = int(row[0:2]),row[2:4]
            feature, data_label = (int(row[0]),int(row[1])),(float(row[2]) * 100 ,float(row[3]) * 100) #convert to cm
            #feature, data_label = (int(row[0]),int(row[1])),(round(float(row[2]) * 100) ,round(float(row[3]) * 100)) #convert to cm
            net(feature)
            eval_data.append([feature, data_label])
            


